<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Building The Prediction Model

### Riccardo Spolaor (864877)

</center>

Modifico il dataframe in modo da poter ottenere un modello adatto alla predizione:
* Rinomino ***Winner*** in ***PlayerA*** e ***Loser*** in ***PlayerB***;
* Rinomino tutte le colonne con informazioni legate al vincitore e al perdente in modo da riflettere il legame ai nuovi nomi assegnati;
* Creo la colonna ***Winner*** che assume i valori:
    * 0 se ha vinto ***PlayerA***;
    * 1 se ha vinto ***PlayerB***;
* Assegno a ***Winner*** tutti valori 0;
* Duplico il dataframe e faccio lo swapping tra tutte le colonne con informazioni sul Giocatore A e quelle con informazioni sul Giocatore B;
* Assegno il valore 1 a ogni entry della colonna ***Winner*** del nuovo dataframe;
* Unisco i due dataframe.

In [85]:
import pandas as pd

In [86]:
dataframe = pd.read_csv('./datasets/dataframe02.csv', low_memory = False)

In [87]:
dataframe.columns


Index(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series', 'Round',
       'Best of', 'Winner', 'Loser',
       ...
       'Kohlschreiber P.A', 'Kohlschreiber P.B', 'Simon G.A', 'Simon G.B',
       'Bautista Agut R.A', 'Bautista Agut R.B', 'Verdasco F.A',
       'Verdasco F.B', 'OtherA', 'OtherB'],
      dtype='object', length=103)

In [88]:
def renameColumns():
    dataframe.rename(columns={'Winner':'PlayerA', 
                          'Loser':'PlayerB',
                          'WRank':'RankA',
                          'WRankAdded':'RankFilledA',
                          'LRank':'RankB',
                          'LRankAdded':'RankFilledB',
                          'W1':'1A',
                          'L1':'1B',
                          'W2':'2A',
                          'L2':'2B',
                          'W3':'3A',
                          'L3':'3B',
                          'W4':'4A',
                          'L4':'4B',
                          'W5':'5A',
                          'L5':'5B',
                          'Wsets':'setsA',
                          'Lsets':'setsB',
                          'WPts':'PtsA',
                          'LPts':'PtsB',
                          'WPtsAdded':'PtsFilledA',
                          'LPtsAdded':'PtsFilledB',
                          'MaxW':'MaxA',
                          'MaxWAdded':'MaxFilledA',
                          'MaxL':'MaxB',
                          'MaxLAdded':'MaxFilledB',
                          'AvgW':'AvgA',
                          'AvgWAdded':'AvgFilledA',
                          'AvgL':'AvgB',
                          'AvgLAdded':'AvgFilledB',
                          'MinW':'MinA',
                          'MinL':'MinB',
                         }, inplace = True)
    dataframe.insert(0, column ='Winner', value = 0)

In [89]:
renameColumns()
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Round', 'Best of', 'PlayerA',
       ...
       'Kohlschreiber P.A', 'Kohlschreiber P.B', 'Simon G.A', 'Simon G.B',
       'Bautista Agut R.A', 'Bautista Agut R.B', 'Verdasco F.A',
       'Verdasco F.B', 'OtherA', 'OtherB'],
      dtype='object', length=104)

In [90]:
def getInvertedDataFrame(colToInvert):
    
    players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:25]
    
    colToInvert += players
    
    dataframeInverted = dataframe.copy(deep = True)

    dataframeInverted['Winner'] = 1
    
    for f in colToInvert:
        dataframeInverted[[f+'A', f+'B']] = dataframeInverted[[f+'B', f+'A']]
    
    return dataframeInverted

In [91]:
dataframeInverted = getInvertedDataFrame(['Player', 'Rank', 'RankFilled', 'Pts', 'PtsFilled', '1', 
                   '2', '3', '4', '5', 'sets', 'Max', 'Avg', 'Min', 'Other', 'MaxFilled', 'AvgFilled'])

In [92]:
dataframeInverted.head()

,Winner,csvID,ATP,Location,Tournament,Date,Series,Round,Best of,PlayerA,...,Kohlschreiber P.A,Kohlschreiber P.B,Simon G.A,Simon G.B,Bautista Agut R.A,Bautista Agut R.B,Verdasco F.A,Verdasco F.B,OtherA,OtherB
0,1,0,1.0,13,0,2010-01-04,0,0,3.0,Nieminen J.,...,0,0,0,0,0,0,0,0,1,0
1,1,0,1.0,13,0,2010-01-04,0,0,3.0,Clement A.,...,0,0,0,0,0,0,0,0,1,1
2,1,0,1.0,13,0,2010-01-04,0,0,3.0,Petzschner P.,...,0,0,0,0,0,0,0,0,1,1
3,1,0,1.0,13,0,2010-01-04,0,0,3.0,Chardy J.,...,0,0,0,0,0,0,0,0,1,1
4,1,0,1.0,13,0,2010-01-04,0,0,3.0,Llodra M.,...,0,0,0,0,0,0,0,0,1,1


In [93]:
dataframe = dataframe.append(dataframeInverted, sort = False, ignore_index = True)

In [94]:
def sortMatchesByDate ():
    dataframe['Index'] = dataframe.index
    dataframe.sort_values(['Date','Index'],inplace = True)
    dataframe.drop('Index', axis=1, inplace = True)
    
sortMatchesByDate()

In [95]:
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Round', 'Best of', 'PlayerA',
       ...
       'Kohlschreiber P.A', 'Kohlschreiber P.B', 'Simon G.A', 'Simon G.B',
       'Bautista Agut R.A', 'Bautista Agut R.B', 'Verdasco F.A',
       'Verdasco F.B', 'OtherA', 'OtherB'],
      dtype='object', length=104)

In [96]:
dataframe.drop(['Date', 'PlayerA', 'PlayerB', '1A', '1B', '2A', '2B', '3A', '3B', '4A', 
                '4B', '5A', '5B', 'setsA', 'setsB', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover'], axis = 1, inplace = True)

In [97]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head()

,Winner,csvID,ATP,Location,Tournament,Series,Round,Best of,RankA,RankFilledA,...,Kohlschreiber P.A,Kohlschreiber P.B,Simon G.A,Simon G.B,Bautista Agut R.A,Bautista Agut R.B,Verdasco F.A,Verdasco F.B,OtherA,OtherB
0,0,0,1.0,13,0,0,0,3.0,52.0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1.0,13,0,0,0,3.0,105.0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,1.0,13,0,0,0,3.0,58.0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,1.0,13,0,0,0,3.0,81.0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,1.0,13,0,0,0,3.0,119.0,0,...,0,0,0,0,0,0,0,0,1,1


In [98]:
dataframe.to_csv('./datasets/dataframe03.csv', index = False)